## Imports, Read data, and Generate Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import requests, json

df = pd.read_csv("./SalaryData.csv")

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

df_copy = train_set.copy()

test_set_full = test_set.copy()
test_set = test_set.drop(["Salary"], axis=1)

train_labels = df_copy["Salary"]

train_set_full = train_set.copy()
train_set = train_set.drop(["Salary"], axis=1)

lin_reg = LinearRegression()

lin_reg.fit(train_set, train_labels)

salary_pred = lin_reg.predict(test_set)

salary_pred

/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


array([115790.21011287,  71498.27809463, 102596.86866063,  75267.80422384,
        55477.79204548,  60189.69970699])

In [13]:
lin_reg.predict([[10.0]])

array([119559.73624209])

## Run Model API

### Persist Model and Data

#### Python Pickle

In [2]:
import pickle

with open("python_lin_reg_model.pkl", "wb") as file_handler:
    pickle.dump(lin_reg, file_handler)
    
with open("python_lin_reg_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### `scikit-learn`'s `joblib`

In [3]:
BASE_URL = "http://localhost:9080"

joblib.dump(lin_reg, "linear_regression_model.pkl")

joblib.dump(train_set, "training_data.pkl")
joblib.dump(train_labels, "training_labels.pkl")

['training_labels.pkl']

### Predict API

In [55]:
years_exp = {"yearsOfExperience": 20}

response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

#response

In [51]:
response

<Response [200]>

In [53]:
response.content

b'[118119.20211752264]'

In [13]:
df_copy.query('YearsExperience > 7 and YearsExperience <= 8')

YearsExperience    Salary
22              7.9  101302.0
21              7.1   98273.0

### Retrain API

In [41]:
data = json.dumps([{"YearsExperience": 40,"Salary": 140000}, 
                   {"YearsExperience": 12.1,"Salary": 142000},
                  {"YearsExperience": 16.1,"Salary": 162000},
                  {"YearsExperience": 24.1,"Salary": 242000},
                  {"YearsExperience": 35.1,"Salary": 542000}])

data

'[{"YearsExperience": 40, "Salary": 140000}, {"YearsExperience": 12.1, "Salary": 142000}, {"YearsExperience": 16.1, "Salary": 162000}, {"YearsExperience": 24.1, "Salary": 242000}, {"YearsExperience": 35.1, "Salary": 542000}]'

In [54]:
response = requests.post("{}/retrain".format(BASE_URL), json = data)

response.json()

{'data': 'Retrained model successfully.'}

In [56]:
response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

response.json()

[195734.81688189803]

### Current Details API

In [57]:
response = requests.get("{}/currentDetails".format(BASE_URL))

response.json()

{'coefficients': [7643.125982668216],
 'intercepts': 42872.29722853372,
 'score': 0.49218459336852394}